In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Load dataset
df = pd.read_csv("/content/Final_reverse_One_hot_encoded_Processed_data.csv")

# Identify all one-hot encoded columns dynamically
target_columns = ['clarity', 'color', 'cut']

# Drop all one-hot encoded target columns
X = df.drop(columns=target_columns)

# OneHotEncode the original target labels
encoder = OneHotEncoder(sparse_output=False)  # ✅ Corrected argument

y_clarity = encoder.fit_transform(df[["clarity"]])
y_color = encoder.fit_transform(df[["color"]])
y_cut = encoder.fit_transform(df[["cut"]])


# Train-test split
X_train, X_test, y_clarity_train, y_clarity_test, y_color_train, y_color_test, y_cut_train, y_cut_test = train_test_split(
    X, y_clarity, y_color, y_cut, test_size=0.4, random_state=42
)

X_val, X_test, y_clarity_val, y_clarity_test, y_color_val, y_color_test, y_cut_val, y_cut_test = train_test_split(
    X_test, y_clarity_test, y_color_test, y_cut_test, test_size=0.5, random_state=42
)


# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert labels to NumPy arrays (without resampling)
Y_clarity_train_array = np.array(y_clarity_train)
Y_color_train_array = np.array(y_color_train)
Y_cut_train_array = np.array(y_cut_train)

Y_clarity_val_array = np.array(y_clarity_val)
Y_color_val_array = np.array(y_color_val)
Y_cut_val_array = np.array(y_cut_val)

Y_clarity_test_array = np.array(y_clarity_test)
Y_color_test_array = np.array(y_color_test)
Y_cut_test_array = np.array(y_cut_test)



# Define the model
input_layer = keras.Input(shape=(X_train_scaled.shape[1],))

# Shared hidden layers
x = layers.Dense(128, activation="relu")(input_layer)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)

# Output layers
clarity_output = layers.Dense(Y_clarity_train_array.shape[1], activation="softmax", name="clarity")(x)
color_output = layers.Dense(Y_color_train_array.shape[1], activation="softmax", name="color")(x)
cut_output = layers.Dense(Y_cut_train_array.shape[1], activation="softmax", name="cut")(x)

# Compile the model
model = keras.Model(inputs=input_layer, outputs=[clarity_output, color_output, cut_output])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'clarity': 'categorical_crossentropy',
        'color': 'categorical_crossentropy',
        'cut': 'categorical_crossentropy'
    },
    metrics={
        'clarity': 'accuracy',
        'color': 'accuracy',
        'cut': 'accuracy'
    }
)

model.summary()

# Train the model
history = model.fit(
    X_train_scaled,
    {"clarity": Y_clarity_train_array, "color": Y_color_train_array, "cut": Y_cut_train_array},
    validation_data=(X_val_scaled, {"clarity": Y_clarity_val_array, "color": Y_color_val_array, "cut": Y_cut_val_array}),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluate the model
Y_test_pred = model.predict(X_test_scaled)
Y_test_pred = [np.argmax(pred, axis=1) for pred in Y_test_pred]  # Convert probabilities to class labels

# Convert one-hot test labels back to class labels
Y_test_true = [
    np.argmax(Y_clarity_test_array, axis=1),
    np.argmax(Y_color_test_array, axis=1),
    np.argmax(Y_cut_test_array, axis=1)
]

# Generate classification reports
for i, name in enumerate(["Clarity", "Color", "Cut"]):
    print(f"{name} Classification Report:\n", classification_report(Y_test_true[i], Y_test_pred[i]))

# Save the model
model.save("DNN_gemstone_quality_model.h5")
print("Model saved successfully.")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 226)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         29,056 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 256)            │         33,024 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 256)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         32,896 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 128)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 64)             │          8,256 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ clarity (Dense)           │ (None, 4)              │            260 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ color (Dense)             │ (None, 28)             │          1,820 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cut (Dense)               │ (None, 14)             │            910 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 106,222 (414.93 KB)

 Trainable params: 106,222 (414.93 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - clarity_accuracy: 0.4370 - clarity_loss: 1.1452 - color_accuracy: 0.2756 - color_loss: 2.1985 - cut_accuracy: 0.3597 - cut_loss: 1.6148 - loss: 4.9586 - val_clarity_accuracy: 0.5943 - val_clarity_loss: 0.7872 - val_color_accuracy: 0.3916 - val_color_loss: 1.6163 - val_cut_accuracy: 0.4255 - val_cut_loss: 1.3672 - val_loss: 3.7715
Epoch 2/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - clarity_accuracy: 0.5837 - clarity_loss: 0.8111 - color_accuracy: 0.3821 - color_loss: 1.6473 - cut_accuracy: 0.4126 - cut_loss: 1.3825 - loss: 3.8408 - val_clarity_accuracy: 0.6180 - val_clarity_loss: 0.7526 - val_color_accuracy: 0.4120 - val_color_loss: 1.5155 - val_cut_accuracy: 0.4373 - val_cut_loss: 1.3483 - val_loss: 3.6171
Epoch 3/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - clarity_accuracy: 0.6024 - clarity_loss: 0.7753 - color_accuracy: 0.3967 - color_loss: 1.5577 - cut_accuracy: 0.4223 - cut_loss: 1.3552 - loss: 3.6882 - val_clarity_accuracy:

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Clarity Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.80      0.75      1665
           1       1.00      1.00      1.00      1708
           2       0.78      0.95      0.85      1631
           3       0.69      0.46      0.56      1710

    accuracy                           0.80      6714
   macro avg       0.80      0.80      0.79      6714
weighted avg       0.80      0.80      0.79      6714

Color Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.79      0.67       949
           1       0.44      0.37      0.40       965
           2       0.00      0.00      0.00       987
           3       0.50      0.85      0.63      1001
           4       0.47      0.83      0.60       972
           5       0.58      0.29      0.39       482
           7       0.00      0.00      0.00        24
           8       0.40      0.26      0.32        23
          10    

# Increase size of layers

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("/content/Final_reverse_One_hot_encoded_Processed_data (1).csv")

# Identify target columns
target_columns = ['clarity', 'color', 'cut']

# Drop target columns to get feature set
X = df.drop(columns=target_columns)

# OneHotEncode target labels
encoder = OneHotEncoder(sparse_output=False)
y_clarity = encoder.fit_transform(df[["clarity"]])
y_color = encoder.fit_transform(df[["color"]])
y_cut = encoder.fit_transform(df[["cut"]])

# Train-test split
X_train, X_test, y_clarity_train, y_clarity_test, y_color_train, y_color_test, y_cut_train, y_cut_test = train_test_split(
    X, y_clarity, y_color, y_cut, test_size=0.4, random_state=42
)

X_val, X_test, y_clarity_val, y_clarity_test, y_color_val, y_color_test, y_cut_val, y_cut_test = train_test_split(
    X_test, y_clarity_test, y_color_test, y_cut_test, test_size=0.5, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert labels to NumPy arrays
Y_clarity_train_array = np.array(y_clarity_train)
Y_color_train_array = np.array(y_color_train)
Y_cut_train_array = np.array(y_cut_train)

Y_clarity_val_array = np.array(y_clarity_val)
Y_color_val_array = np.array(y_color_val)
Y_cut_val_array = np.array(y_cut_val)

Y_clarity_test_array = np.array(y_clarity_test)
Y_color_test_array = np.array(y_color_test)
Y_cut_test_array = np.array(y_cut_test)

# Define the model with increased layer sizes
input_layer = keras.Input(shape=(X_train_scaled.shape[1],))

# Deepened and widened hidden layers
x = layers.Dense(512, activation="relu")(input_layer)
x = layers.Dropout(0.4)(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(128, activation="relu")(x)

# Output layers
clarity_output = layers.Dense(Y_clarity_train_array.shape[1], activation="softmax", name="clarity")(x)
color_output = layers.Dense(Y_color_train_array.shape[1], activation="softmax", name="color")(x)
cut_output = layers.Dense(Y_cut_train_array.shape[1], activation="softmax", name="cut")(x)

# Compile the model
model = keras.Model(inputs=input_layer, outputs=[clarity_output, color_output, cut_output])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={'clarity': 'categorical_crossentropy', 'color': 'categorical_crossentropy', 'cut': 'categorical_crossentropy'},
    metrics={'clarity': 'accuracy', 'color': 'accuracy', 'cut': 'accuracy'}
)

model.summary()

# Train the model
history = model.fit(
    X_train_scaled,
    {"clarity": Y_clarity_train_array, "color": Y_color_train_array, "cut": Y_cut_train_array},
    validation_data=(X_val_scaled, {"clarity": Y_clarity_val_array, "color": Y_color_val_array, "cut": Y_cut_val_array}),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluate the model
Y_test_pred = model.predict(X_test_scaled)
Y_test_pred = [np.argmax(pred, axis=1) for pred in Y_test_pred]  # Convert probabilities to class labels

# Convert one-hot test labels back to class labels
Y_test_true = [
    np.argmax(Y_clarity_test_array, axis=1),
    np.argmax(Y_color_test_array, axis=1),
    np.argmax(Y_cut_test_array, axis=1)
]

# Generate classification reports
for i, name in enumerate(["Clarity", "Color", "Cut"]):
    print(f"{name} Classification Report:\n", classification_report(Y_test_true[i], Y_test_pred[i]))

# Save the model
model.save("DNN_gemstone_quality_model_large.h5")
print("Model with increased layers saved successfully.")


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 226)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 512)            │        116,224 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 512)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 512)            │        262,656 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 512)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 256)            │        131,328 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 256)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 128)            │         32,896 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ clarity (Dense)           │ (None, 4)              │            516 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ color (Dense)             │ (None, 28)             │          3,612 │ dense_3[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cut (Dense)               │ (None, 14)             │          1,806 │ dense_3[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 549,038 (2.09 MB)

 Trainable params: 549,038 (2.09 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - clarity_accuracy: 0.4530 - clarity_loss: 1.1152 - color_accuracy: 0.2820 - color_loss: 2.1560 - cut_accuracy: 0.3509 - cut_loss: 1.5876 - loss: 4.8588 - val_clarity_accuracy: 0.5895 - val_clarity_loss: 0.7893 - val_color_accuracy: 0.3765 - val_color_loss: 1.6101 - val_cut_accuracy: 0.4242 - val_cut_loss: 1.3847 - val_loss: 3.7847
Epoch 2/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - clarity_accuracy: 0.5705 - clarity_loss: 0.8259 - color_accuracy: 0.3729 - color_loss: 1.6423 - cut_accuracy: 0.4044 - cut_loss: 1.3936 - loss: 3.8618 - val_clarity_accuracy: 0.6029 - val_clarity_loss: 0.7659 - val_color_accuracy: 0.4074 - val_color_loss: 1.5287 - val_cut_accuracy: 0.4225 - val_cut_loss: 1.3633 - val_loss: 3.6586
Epoch 3/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - clarity_accuracy: 0.5955 - clarity_loss: 0.7844 - color_accuracy: 0.3947 - color_loss: 1.5538 - cut_accuracy: 0.4236 - cut_loss: 1.3486 - loss: 3.6868 - val_clarity_accura

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Clarity Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.85      0.76      1665
           1       1.00      1.00      1.00      1708
           2       0.78      0.95      0.86      1631
           3       0.73      0.43      0.54      1710

    accuracy                           0.80      6714
   macro avg       0.80      0.81      0.79      6714
weighted avg       0.80      0.80      0.79      6714

Color Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.68      0.63       949
           1       0.45      0.42      0.43       965
           2       0.00      0.00      0.00       987
           3       0.50      0.85      0.63      1001
           4       0.46      0.87      0.61       972
           5       0.63      0.22      0.33       482
           7       0.00      0.00      0.00        24
           8       0.20      0.04      0.07        23
          10    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("/content/Final_reverse_One_hot_encoded_Processed_data (1).csv")

# Identify target columns
target_columns = ['clarity', 'color', 'cut']

# Drop target columns to get feature set
X = df.drop(columns=target_columns)

# OneHotEncode target labels
encoder = OneHotEncoder(sparse_output=False)
y_clarity = encoder.fit_transform(df[["clarity"]])
y_color = encoder.fit_transform(df[["color"]])
y_cut = encoder.fit_transform(df[["cut"]])

# Train-test split
X_train, X_test, y_clarity_train, y_clarity_test, y_color_train, y_color_test, y_cut_train, y_cut_test = train_test_split(
    X, y_clarity, y_color, y_cut, test_size=0.4, random_state=42
)

X_val, X_test, y_clarity_val, y_clarity_test, y_color_val, y_color_test, y_cut_val, y_cut_test = train_test_split(
    X_test, y_clarity_test, y_color_test, y_cut_test, test_size=0.5, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert labels to NumPy arrays
Y_clarity_train_array = np.array(y_clarity_train)
Y_color_train_array = np.array(y_color_train)
Y_cut_train_array = np.array(y_cut_train)

Y_clarity_val_array = np.array(y_clarity_val)
Y_color_val_array = np.array(y_color_val)
Y_cut_val_array = np.array(y_cut_val)

Y_clarity_test_array = np.array(y_clarity_test)
Y_color_test_array = np.array(y_color_test)
Y_cut_test_array = np.array(y_cut_test)


# Define the model with larger layers and added linear layers
input_layer = keras.Input(shape=(X_train_scaled.shape[1],))

# First linear layer before activation
x = layers.Dense(256, activation="linear")(input_layer)  # Linear transformation
x = layers.Dense(512, activation="relu")(x)  # Increased layer size
x = layers.Dropout(0.3)(x)
x = layers.Dense(1024, activation="relu")(x)  # Larger hidden layer
x = layers.Dense(512, activation="linear")(x)  # Another linear transformation
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)

# Output layers (remains the same)
clarity_output = layers.Dense(Y_clarity_train_array.shape[1], activation="softmax", name="clarity")(x)
color_output = layers.Dense(Y_color_train_array.shape[1], activation="softmax", name="color")(x)
cut_output = layers.Dense(Y_cut_train_array.shape[1], activation="softmax", name="cut")(x)

# Compile the model
model = keras.Model(inputs=input_layer, outputs=[clarity_output, color_output, cut_output])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'clarity': 'categorical_crossentropy',
        'color': 'categorical_crossentropy',
        'cut': 'categorical_crossentropy'
    },
    metrics={
        'clarity': 'accuracy',
        'color': 'accuracy',
        'cut': 'accuracy'
    }
)



model.summary()

# Train the model
history = model.fit(
    X_train_scaled,
    {"clarity": Y_clarity_train_array, "color": Y_color_train_array, "cut": Y_cut_train_array},
    validation_data=(X_val_scaled, {"clarity": Y_clarity_val_array, "color": Y_color_val_array, "cut": Y_cut_val_array}),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluate the model
Y_test_pred = model.predict(X_test_scaled)
Y_test_pred = [np.argmax(pred, axis=1) for pred in Y_test_pred]  # Convert probabilities to class labels

# Convert one-hot test labels back to class labels
Y_test_true = [
    np.argmax(Y_clarity_test_array, axis=1),
    np.argmax(Y_color_test_array, axis=1),
    np.argmax(Y_cut_test_array, axis=1)
]

# Generate classification reports
for i, name in enumerate(["Clarity", "Color", "Cut"]):
    print(f"{name} Classification Report:\n", classification_report(Y_test_true[i], Y_test_pred[i]))

# Save the model
model.save("DNN_gemstone_quality_model_large.h5")
print("Model with increased layers saved successfully.")

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 226)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 256)            │         58,112 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 512)            │        131,584 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 512)            │              0 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 1024)           │        525,312 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 512)            │        524,800 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 512)            │              0 │ dense_7[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 256)            │        131,328 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 256)            │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 128)            │         32,896 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ clarity (Dense)           │ (None, 4)              │            516 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ color (Dense)             │ (None, 28)             │          3,612 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cut (Dense)               │ (None, 14)             │          1,806 │ dense_9[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,409,966 (5.38 MB)

 Trainable params: 1,409,966 (5.38 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 18s 25ms/step - clarity_accuracy: 0.5007 - clarity_loss: 1.0203 - color_accuracy: 0.3086 - color_loss: 2.0168 - cut_accuracy: 0.3645 - cut_loss: 1.5592 - loss: 4.5963 - val_clarity_accuracy: 0.6046 - val_clarity_loss: 0.7666 - val_color_accuracy: 0.4220 - val_color_loss: 1.5298 - val_cut_accuracy: 0.4245 - val_cut_loss: 1.3459 - val_loss: 3.6431
Epoch 2/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - clarity_accuracy: 0.5967 - clarity_loss: 0.7821 - color_accuracy: 0.3985 - color_loss: 1.5682 - cut_accuracy: 0.4217 - cut_loss: 1.3551 - loss: 3.7054 - val_clarity_accuracy: 0.6306 - val_clarity_loss: 0.7452 - val_color_accuracy: 0.4214 - val_color_loss: 1.4722 - val_cut_accuracy: 0.4330 - val_cut_loss: 1.3245 - val_loss: 3.5426
Epoch 3/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - clarity_accuracy: 0.6283 - clarity_loss: 0.7463 - color_accuracy: 0.4194 - color_loss: 1.4932 - cut_accuracy: 0.4354 - cut_loss: 1.3181 - loss: 3.5577 - val_clarity_acc

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Clarity Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.83      0.79      1665
           1       1.00      1.00      1.00      1708
           2       0.86      0.96      0.91      1631
           3       0.76      0.58      0.66      1710

    accuracy                           0.84      6714
   macro avg       0.84      0.84      0.84      6714
weighted avg       0.84      0.84      0.84      6714

Color Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.63      0.66       949
           1       0.47      0.67      0.55       965
           2       0.00      0.00      0.00       987
           3       0.50      0.85      0.63      1001
           4       0.49      0.89      0.63       972
           5       0.62      0.20      0.30       482
           7       0.00      0.00      0.00        24
           8       0.50      0.04      0.08        23
          10    

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report

# Load dataset
df = pd.read_csv("/content/Final_reverse_One_hot_encoded_Processed_data (1).csv")

# Identify target columns
target_columns = ['clarity', 'color', 'cut']

# Drop target columns to get feature set
X = df.drop(columns=target_columns)

# OneHotEncode target labels
encoder = OneHotEncoder(sparse_output=False)
y_clarity = encoder.fit_transform(df[["clarity"]])
y_color = encoder.fit_transform(df[["color"]])
y_cut = encoder.fit_transform(df[["cut"]])

# Train-test split
X_train, X_test, y_clarity_train, y_clarity_test, y_color_train, y_color_test, y_cut_train, y_cut_test = train_test_split(
    X, y_clarity, y_color, y_cut, test_size=0.4, random_state=42
)

X_val, X_test, y_clarity_val, y_clarity_test, y_color_val, y_color_test, y_cut_val, y_cut_test = train_test_split(
    X_test, y_clarity_test, y_color_test, y_cut_test, test_size=0.5, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Convert labels to NumPy arrays
Y_clarity_train_array = np.array(y_clarity_train)
Y_color_train_array = np.array(y_color_train)
Y_cut_train_array = np.array(y_cut_train)

Y_clarity_val_array = np.array(y_clarity_val)
Y_color_val_array = np.array(y_color_val)
Y_cut_val_array = np.array(y_cut_val)

Y_clarity_test_array = np.array(y_clarity_test)
Y_color_test_array = np.array(y_color_test)
Y_cut_test_array = np.array(y_cut_test)


# Define the model with larger layers and added linear layers
input_layer = keras.Input(shape=(X_train_scaled.shape[1],))


# First linear layer before activation
x = layers.Dense(256, activation="linear")(input_layer)  # Linear transformation
x = layers.Dense(512, activation="relu")(x)  # Increased layer size
x = layers.Dropout(0.3)(x)
x = layers.Dense(1024, activation="relu")(x)  # Larger hidden layer
x = layers.Dense(512, activation="linear")(x)  # Another linear transformation
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
# Add a 64-unit layer after the 128-unit layer
x = layers.Dense(64, activation="relu")(x)



# Output layers (remains the same)
clarity_output = layers.Dense(Y_clarity_train_array.shape[1], activation="softmax", name="clarity")(x)
color_output = layers.Dense(Y_color_train_array.shape[1], activation="softmax", name="color")(x)
cut_output = layers.Dense(Y_cut_train_array.shape[1], activation="softmax", name="cut")(x)


# Compile the model
model = keras.Model(inputs=input_layer, outputs=[clarity_output, color_output, cut_output])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss={
        'clarity': 'categorical_crossentropy',
        'color': 'categorical_crossentropy',
        'cut': 'categorical_crossentropy'
    },
    metrics={
        'clarity': 'accuracy',
        'color': 'accuracy',
        'cut': 'accuracy'
    }
)



model.summary()

# Train the model
history = model.fit(
    X_train_scaled,
    {"clarity": Y_clarity_train_array, "color": Y_color_train_array, "cut": Y_cut_train_array},
    validation_data=(X_val_scaled, {"clarity": Y_clarity_val_array, "color": Y_color_val_array, "cut": Y_cut_val_array}),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Evaluate the model
Y_test_pred = model.predict(X_test_scaled)
Y_test_pred = [np.argmax(pred, axis=1) for pred in Y_test_pred]  # Convert probabilities to class labels

# Convert one-hot test labels back to class labels
Y_test_true = [
    np.argmax(Y_clarity_test_array, axis=1),
    np.argmax(Y_color_test_array, axis=1),
    np.argmax(Y_cut_test_array, axis=1)
]

# Generate classification reports
for i, name in enumerate(["Clarity", "Color", "Cut"]):
    print(f"{name} Classification Report:\n", classification_report(Y_test_true[i], Y_test_pred[i]))

# Save the model
model.save("DNN_gemstone_quality_model_large.h5")
print("Model with increased layers saved successfully.")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 226)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 256)            │         58,112 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 512)            │        131,584 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 512)            │              0 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 1024)           │        525,312 │ dropout_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 512)            │        524,800 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 512)            │              0 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 256)            │        131,328 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 256)            │              0 │ dense_14[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 128)            │         32,896 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 64)             │          8,256 │ dense_15[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ clarity (Dense)           │ (None, 4)              │            260 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ color (Dense)             │ (None, 28)             │          1,820 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cut (Dense)               │ (None, 14)             │            910 │ dense_16[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,415,278 (5.40 MB)

 Trainable params: 1,415,278 (5.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - clarity_accuracy: 0.4916 - clarity_loss: 1.0303 - color_accuracy: 0.3018 - color_loss: 2.0893 - cut_accuracy: 0.3721 - cut_loss: 1.5557 - loss: 4.6753 - val_clarity_accuracy: 0.5952 - val_clarity_loss: 0.7829 - val_color_accuracy: 0.3996 - val_color_loss: 1.5999 - val_cut_accuracy: 0.4285 - val_cut_loss: 1.3825 - val_loss: 3.7658
Epoch 2/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - clarity_accuracy: 0.5981 - clarity_loss: 0.7894 - color_accuracy: 0.3918 - color_loss: 1.5852 - cut_accuracy: 0.4187 - cut_loss: 1.3634 - loss: 3.7381 - val_clarity_accuracy: 0.6032 - val_clarity_loss: 0.7740 - val_color_accuracy: 0.4051 - val_color_loss: 1.5375 - val_cut_accuracy: 0.3996 - val_cut_loss: 1.3563 - val_loss: 3.6685
Epoch 3/50
630/630 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - clarity_accuracy: 0.6135 - clarity_loss: 0.7571 - color_accuracy: 0.4086 - color_loss: 1.5072 - cut_accuracy: 0.4308 - cut_loss: 1.3313 - loss: 3.5957 - val_clarity_acc

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

Clarity Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.77      0.79      1665
           1       1.00      1.00      1.00      1708
           2       0.84      0.97      0.90      1631
           3       0.75      0.67      0.71      1710

    accuracy                           0.85      6714
   macro avg       0.85      0.85      0.85      6714
weighted avg       0.85      0.85      0.85      6714

Color Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.75      0.68       949
           1       0.48      0.39      0.43       965
           2       0.00      0.00      0.00       987
           3       0.50      0.85      0.63      1001
           4       0.49      0.92      0.63       972
           5       0.69      0.19      0.30       482
           7       0.00      0.00      0.00        24
           8       0.45      0.22      0.29        23
          10    